In [2]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [3]:
DB_HOST = 'localhost'  
DB_PORT = '5433'      
DB_NAME = 'taxi_data'  
DB_USER = 'postgres'  
DB_PASSWORD = 'admin' 

print(f"database: Host={DB_HOST}, Port={DB_PORT}, DB={DB_NAME}")

database: Host=localhost, Port=5433, DB=taxi_data


In [4]:
# creating the databse
def create_database():
    try:
        # connecting to PostgreSQL server
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password=DB_PASSWORD
        )
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        # checking if database exists
        cursor.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{DB_NAME}'")
        exists = cursor.fetchone()
        
        if not exists:
            cursor.execute(f'CREATE DATABASE {DB_NAME}')
            print(f"database {DB_NAME} created successfully")
        else:
            print(f"Database {DB_NAME} already exists")
            
        cursor.close()
        conn.close()
        return True
    except Exception as e:
        print(f"error creating database: {e}")
        return False

db_created = create_database()
db_created

Database taxi_data already exists


True

In [5]:
# tables
def create_tables(engine):
    try:
        create_table_query = """
        CREATE TABLE IF NOT EXISTS taxi_trips (
            id SERIAL PRIMARY KEY,
            vendorid VARCHAR(10),
            tpep_pickup_datetime TIMESTAMP,
            tpep_dropoff_datetime TIMESTAMP,
            passenger_count NUMERIC,
            trip_distance NUMERIC,
            ratecodeid NUMERIC,
            pulocationid VARCHAR(10),
            dolocationid VARCHAR(10),
            payment_type VARCHAR(10),
            fare_amount NUMERIC(10, 2),
            extra NUMERIC(10, 2),
            mta_tax NUMERIC(10, 2),
            tip_amount NUMERIC(10, 2),
            tolls_amount NUMERIC(10, 2),
            improvement_surcharge NUMERIC(10, 2),
            total_amount NUMERIC(10, 2),
            congestion_surcharge NUMERIC(10, 2),
            airport_fee NUMERIC(10, 2)
        );
        """
        
        with engine.connect() as conn:
            conn.execute(text(create_table_query))
            conn.commit()
            print("table 'taxi_trips' created")
        return True
            
    except Exception as e:
        print(f"error creating tables: {e}")
        return False

# SQLAlchemy
engine_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(engine_url)

# tables
tables_created = create_tables(engine)
tables_created

table 'taxi_trips' created


True

In [6]:
# loading cleaned data
df = pd.read_csv('data/cleaned_taxi_data.csv')
print(f"{len(df)} records from CSV file")

df.head()

1000 records from CSV file


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01T00:32:10.000,2023-01-01T00:40:36.000,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01T00:55:08.000,2023-01-01T01:01:27.000,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01T00:25:04.000,2023-01-01T00:37:49.000,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01T00:03:48.000,2023-01-01T00:13:25.000,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,22.10,0.0,1.25
4,2,2023-01-01T00:10:29.000,2023-01-01T00:21:19.000,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
# converting date and time cols to datetime format
if 'tpep_pickup_datetime' in df.columns:
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
if 'tpep_dropoff_datetime' in df.columns:
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df.dtypes

vendorid                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
ratecodeid                      float64
store_and_fwd_flag               object
pulocationid                      int64
dolocationid                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [8]:
# loading data to PostgreSQL
try:
    df.to_sql('taxi_trips', engine, if_exists='replace', index=False)
    
    print(f"loaded {len(df)} records to PostgreSQL")
    success = True
except Exception as e:
    print(f"error loading data to PostgreSQL: {e}")
    success = False

success

loaded 1000 records to PostgreSQL


True

In [12]:
try:
    # get row count
    query = "SELECT COUNT(*) FROM taxi_trips"
    row_count = pd.read_sql(query, engine)
    print(f"num of rows in database: {row_count.iloc[0, 0]}")
    
    # sample data
    query = "SELECT * FROM taxi_trips LIMIT 5"
    sample = pd.read_sql(query, engine)
    print("sample data from database:")
    print(sample)
except Exception as e:
    print(f"error querying database: {e}")

num of rows in database: 1000
sample data from database:
   vendorid tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  ratecodeid store_and_fwd_flag  pulocationid  dolocationid  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  